# SQL Database Agent

This notebook showcases an agent designed to interact with a sql databases. The agent builds off of [SQLDatabaseChain](https://langchain.readthedocs.io/en/latest/modules/chains/examples/sqlite.html) and is designed to answer more general questions about a database, as well as recover from errors.

Note that, as this agent is in active development, all answers might not be correct. Additionally, it is not guaranteed that the agent won't perform DML statements on your database given certain questions. Be careful running it on sensitive data!

This uses the example Chinook database. To set it up follow the instructions on https://database.guide/2-sample-databases-sqlite/, placing the .db file in a notebooks folder at the root of this repository.

## Initialization

In [8]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor

In [9]:
db = SQLDatabase.from_uri("sqlite:///../db/sample.db")
toolkit = SQLDatabaseToolkit(db=db)

agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True
)

## Example: Show erc20 transaction type

In [7]:
agent_executor.run("Show me erc20 type transaction hash")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: transactions
Thought: I should check the schema of the transactions table to see what columns I can query.
Action: schema_sql_db
Action Input: "transactions"
Observation: 
CREATE TABLE transactions (
	transaction_hash TEXT, 
	category TEXT
)
/*
3 rows from transactions table:
transaction_hash	category
0x2e375498d25245599b4ba6ecb7c699af28d1a6d5cfc80e6e838a7d38e480ce1f	external
0xe883393f0f6050cc191eca8b22e7259d1c2777d340a758f2162299a56067788a	erc20
0x55cb39ac28f5a7f3b54a1c03790fc0d3f0bb9da2091d4c19fcb9388ac89565e7	erc20
*/
Thought: I should query the transactions table for erc20 type transaction hashes.
Action: query_sql_db
Action Input: SELECT transaction_hash FROM transactions WHERE category = 'erc20' LIMIT 10
Observation: [('0xe883393f0f6050cc191eca8b22e7259d1c2777d340a758f2162299a56067788a',), ('0x55cb39ac28f5a7f3b54a1c03790fc0d3f0bb9da2091d4c19fcb9388ac89565e7',), ('0x018dff402631c0755d

'0xe883393f0f6050cc191eca8b22e7259d1c2777d340a758f2162299a56067788a, 0x55cb39ac28f5a7f3b54a1c03790fc0d3f0bb9da2091d4c19fcb9388ac89565e7, 0x018dff402631c0755de56084905a636615b3fb136d5748d3441148c8b5370d9a, 0xb5f022d930f28b9211826c9dfbf1a6f876079418afd49a6bb1dab3f7d8d091ec, 0xbf09d33e3a5ae21624be1855e37d37dde911f62b0019d4d91b2dae0837237dc4, 0x486d126a34660fdb055e1c0c42d3820a5df7ba61233e1596df1f5c'

## Example: describing a table, recovering from an error

In this example, the agent tries to search for a table that doesn't exist, but finds the next best result

In [10]:
agent_executor.run("Show me USDC transactions")

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).




> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: transactions
Thought: I should check the schema of the transactions table to see what columns I can query.
Action: schema_sql_db
Action Input: "transactions"
Observation: 
CREATE TABLE transactions (
	transaction_hash TEXT, 
	category TEXT, 
	asset TEXT
)
/*
3 rows from transactions table:
transaction_hash	category	asset
0x2e375498d25245599b4ba6ecb7c699af28d1a6d5cfc80e6e838a7d38e480ce1f	external	ETH
0xe883393f0f6050cc191eca8b22e7259d1c2777d340a758f2162299a56067788a	erc20	USDC
0x55cb39ac28f5a7f3b54a1c03790fc0d3f0bb9da2091d4c19fcb9388ac89565e7	erc20	aUSDC
*/
Thought: I should query the transactions table for USDC transactions.
Action: query_sql_db
Action Input: SELECT transaction_hash, category, asset FROM transactions WHERE asset = 'USDC' LIMIT 10
Observation: [('0xe883393f0f6050cc191eca8b22e7259d1c2777d340a758f2162299a56067788a', 'erc20', 'USDC'), ('0xb5f022d930f28b9211826c9dfbf1a6f87607941

'Here are the top 10 USDC transactions: 0xe883393f0f6050cc191eca8b22e7259d1c2777d340a758f2162299a56067788a, 0xb5f022d930f28b9211826c9dfbf1a6f876079418afd49a6bb1dab3f7d8d091ec, 0x2742f6b154c61b25e0f24f08183d3cc7b60fe2c5cc9c793a0f7266007ee838b3, 0x9061fb59faaa5893990f47ad08f88e0f48adffaa72bfbe5036fd86b226fbcdc6, 0xc46d43d8a08acffca734091eea573ccf0f7817d7b2c0bd7b706b12429b2b5d0b, 0x486d126a34660fdb055e1c0c42d3820a5df7ba'

## Example: running queries

In [8]:
agent_executor.run("List the total sales per country. Which country's customers spent the most?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Invoice, MediaType, Artist, InvoiceLine, Genre, Playlist, Employee, Album, PlaylistTrack, Track, Customer
Thought: I should look at the schema of the relevant tables to see what columns I can use.
Action: schema_sql_db
Action Input: "Invoice, Customer"
Observation: 
CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

SELECT * FROM 'Customer' LIMIT 3;
CustomerId FirstName LastName Company Address City State Country PostalCode Phone Fax Email SupportRepId
1 Luís 

'The customers from the USA spent the most, with a total of $523.06.'

In [7]:
agent_executor.run("Show the total number of tracks in each playlist. The Playlist name should be included in the result.")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: Invoice, MediaType, Artist, InvoiceLine, Genre, Playlist, Employee, Album, PlaylistTrack, Track, Customer
Thought: I should look at the schema of the Playlist and PlaylistTrack tables to see what columns I can use.
Action: schema_sql_db
Action Input: "Playlist, PlaylistTrack"
Observation: 
CREATE TABLE "Playlist" (
	"PlaylistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("PlaylistId")
)

SELECT * FROM 'Playlist' LIMIT 3;
PlaylistId Name
1 Music
2 Movies
3 TV Shows


CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

SELECT * FROM 'PlaylistTrack' LIMIT 3;
PlaylistId TrackId
1 3402
1 3389
1 3390
Thought: I can use a SELECT statement to get the total number of tracks in each 

"The total number of tracks in each playlist are: '90’s Music' (1477), 'Brazilian Music' (39), 'Classical' (75), 'Classical 101 - Deep Cuts' (25), 'Classical 101 - Next Steps' (25), 'Classical 101 - The Basics' (25), 'Grunge' (15), 'Heavy Metal Classic' (26), 'Music' (6580), 'Music Videos' (1)."

## Recovering from an error

In this example, the agent is able to recover from an error after initially trying to access an attribute (`Track.ArtistId`) which doesn't exist.

In [16]:
agent_executor.run("Who are the top 3 best selling artists?")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: MediaType, Track, Invoice, Album, Playlist, Customer, Employee, InvoiceLine, PlaylistTrack, Genre, Artist
Thought: I should look at the schema of the Artist, InvoiceLine, and Track tables to see what columns I can use.
Action: schema_sql_db
Action Input: "Artist, InvoiceLine, Track"
Observation: 
CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

SELECT * FROM 'Artist' LIMIT 3;
ArtistId Name
1 AC/DC
2 Accept
3 Aerosmith


CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), 
	FOREIGN KEY("GenreId") REFERENCES "Genre

'The top 3 best selling artists are Iron Maiden, U2, and Metallica.'